In [ ]:
"""
Sistema de Interconsulta Médica Inteligente con LangChain
Manager analiza el caso y DECIDE:
1. Si puede resolverlo solo (caso simple)
2. Qué especialistas necesita consultar (caso complejo)
3. Extrae información pertinente SOLO para los especialistas necesarios
"""

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from typing import List, Dict, Tuple, Optional
import json

class SpecialistAgent:
    """Agente especialista médico"""
    
    def __init__(self, llm: ChatOpenAI, specialty: str, system_prompt: str):
        self.llm = llm
        self.specialty = specialty
        self.system_prompt = system_prompt
        
    def consult(self, specific_question: str, relevant_data: str) -> str:
        """Procesa una consulta específica del área"""
        prompt = f"""{self.system_prompt}

CONSULTA ESPECÍFICA DE {self.specialty.upper()}:
{specific_question}

DATOS CLÍNICOS RELEVANTES:
{relevant_data}

Por favor, proporciona tu interconsulta en formato médico estándar."""

        return self.llm.predict(prompt)


class IntelligentManagerAgent:
    """Manager inteligente que decide si necesita especialistas"""
    
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm
        
    def analyze_case_complexity(self, question: str, patient_record: str) -> Dict:
        """
        Analiza el caso y decide:
        1. Si puede resolverlo como internista
        2. Qué especialistas necesita consultar
        3. Por qué necesita cada especialista
        
        Returns:
            Dict con decisión y justificación
        """
        prompt = f"""Eres un médico internista experimentado y coordinador de interconsultas.

Tu responsabilidad es ANALIZAR cada caso y DECIDIR si:
A) Puedes resolverlo TÚ MISMO como internista (casos de baja-media complejidad)
B) Necesitas consultar ESPECIALISTAS (casos complejos o con datos de alarma)

CASO CLÍNICO:
Pregunta: {question}

Expediente:
{patient_record}

INSTRUCCIONES:
1. Analiza la complejidad del caso
2. Identifica hallazgos que requieran evaluación especializada
3. Decide qué hacer

CRITERIOS PARA CONSULTAR ESPECIALISTAS:
- Neurología: Alteraciones neurológicas específicas, neuropatías, eventos cerebrovasculares, cefaleas complejas
- Cardiología: Cardiopatías, arritmias, insuficiencia cardíaca, síndrome coronario, riesgo CV muy alto
- Endocrinología: Diabetes descompensada, alteraciones tiroideas complejas, trastornos hormonales
- Nefrología: ERC avanzada (G3b o mayor), síndrome nefrótico, glomerulopatías
- Oftalmología: Retinopatía, pérdida visual, alteraciones fundoscópicas importantes

NO CONSULTES si:
- El caso es de complejidad baja-media y está dentro del manejo de medicina interna
- Los hallazgos son inespecíficos o no requieren evaluación urgente
- Puedes manejar el caso con estudios básicos y tratamiento estándar

Responde ÚNICAMENTE en formato JSON:
{{
    "puede_resolver_solo": true/false,
    "complejidad": "baja/media/alta",
    "justificacion_decision": "Explicación de por qué puedes o no resolverlo solo",
    "especialistas_necesarios": ["especialidad1", "especialidad2"] o [],
    "razon_cada_especialista": {{
        "especialidad1": "Por qué es necesario este especialista",
        "especialidad2": "Por qué es necesario este especialista"
    }},
    "hallazgos_criticos": ["hallazgo1", "hallazgo2"]
}}

IMPORTANTE: Solo consulta especialistas cuando sea REALMENTE necesario. Como internista puedes manejar muchos casos.
"""
        response = self.llm.predict(prompt)
        
        # Parsear JSON
        try:
            response = self._clean_json_response(response)
            decision = json.loads(response)
            return decision
        except json.JSONDecodeError as e:
            print(f"⚠️  Error parseando decisión: {e}")
            # Fallback conservador: no consultar especialistas
            return {
                "puede_resolver_solo": True,
                "complejidad": "media",
                "justificacion_decision": "Error en análisis, procederé como internista",
                "especialistas_necesarios": [],
                "razon_cada_especialista": {},
                "hallazgos_criticos": []
            }
    
    def resolve_as_internist(self, question: str, patient_record: str, analysis: Dict) -> str:
        """
        Resuelve el caso directamente como internista
        """
        prompt = f"""Eres un médico internista experimentado. Has analizado un caso y determinaste que PUEDES resolverlo sin interconsultas.

PREGUNTA:
{question}

EXPEDIENTE:
{patient_record}

TU ANÁLISIS PREVIO:
- Complejidad: {analysis['complejidad']}
- Justificación: {analysis['justificacion_decision']}
- Hallazgos críticos: {', '.join(analysis['hallazgos_criticos']) if analysis['hallazgos_criticos'] else 'Ninguno'}

TAREA:
Elabora una NOTA MÉDICA DE CONSULTA INTERNA completa:

NOTA MÉDICA - MEDICINA INTERNA
Fecha: [fecha actual]

1. MOTIVO DE CONSULTA
[Pregunta original]

2. RESUMEN CLÍNICO
[Síntesis del caso con datos relevantes]

3. ANÁLISIS DEL CASO
[Tu evaluación médica integral]

4. IMPRESIÓN DIAGNÓSTICA
[Diagnósticos en orden de probabilidad]
1. Diagnóstico principal
2. Diagnósticos diferenciales

5. FUNDAMENTACIÓN DIAGNÓSTICA
[Justificación clínica de tus diagnósticos]

6. PLAN DE MANEJO
A) Medidas generales
B) Tratamiento farmacológico
C) Estudios complementarios solicitados
D) Modificaciones al estilo de vida
E) Metas terapéuticas

7. PRONÓSTICO
[Evaluación del pronóstico]

8. SEGUIMIENTO
[Plan de seguimiento y criterios de interconsulta futura]

Dr. Roberto Mendoza García
Medicina Interna - Cédula: 1234567
"""
        return self.llm.predict(prompt)
    
    def extract_relevant_info(self, question: str, patient_record: str, 
                             specialty: str, reason: str) -> Tuple[str, str]:
        """
        Extrae información relevante del expediente para un especialista específico
        """
        prompt = f"""Eres un médico internista coordinando una interconsulta.

CASO COMPLETO:
{patient_record}

MOTIVO DE CONSULTA ORIGINAL:
{question}

VAS A SOLICITAR INTERCONSULTA A: {specialty}
RAZÓN ESPECÍFICA: {reason}

TAREA:
Extrae SOLO la información PERTINENTE para {specialty} y formula una pregunta específica.

Responde en formato JSON:
{{
    "pregunta_especifica": "Pregunta concreta para el especialista",
    "datos_relevantes": "Datos del expediente pertinentes SOLO para esta especialidad"
}}

IMPORTANTE:
- Solo incluye antecedentes, síntomas, signos y laboratorios relevantes para {specialty}
- No incluyas información de otras especialidades
- La pregunta debe ser específica y concreta
"""
        response = self.llm.predict(prompt)
        
        try:
            response = self._clean_json_response(response)
            data = json.loads(response)
            return data["pregunta_especifica"], data["datos_relevantes"]
        except json.JSONDecodeError:
            return f"Valoración desde perspectiva de {specialty}: {reason}", patient_record
    
    def integrate_consultations(self, 
                               original_question: str,
                               patient_record: str,
                               analysis: Dict,
                               consultations: Dict[str, str]) -> str:
        """
        Integra todas las interconsultas recibidas
        """
        consultations_text = "\n\n".join([
            f"{'='*60}\n{specialty.upper()}\n{'='*60}\n{response}"
            for specialty, response in consultations.items()
        ])
        
        prompt = f"""Eres el médico internista coordinador. Solicitaste interconsultas a especialistas.

CASO ORIGINAL:
Pregunta: {original_question}

Expediente completo:
{patient_record}

TU ANÁLISIS INICIAL:
Complejidad: {analysis['complejidad']}
Especialistas consultados: {', '.join(consultations.keys())}
Razones: {json.dumps(analysis['razon_cada_especialista'], indent=2, ensure_ascii=False)}

INTERCONSULTAS RECIBIDAS:
{consultations_text}

TAREA:
Elabora una NOTA DE INTEGRACIÓN MÉDICA completa:

NOTA DE INTEGRACIÓN - MEDICINA INTERNA
Fecha: [fecha actual]
Especialistas consultados: {', '.join(consultations.keys())}

1. CONTEXTO
[Breve resumen de por qué se solicitaron interconsultas]

2. SÍNTESIS DEL CASO
[Resumen ejecutivo del paciente]

3. ANÁLISIS DE INTERCONSULTAS

{chr(10).join([f"{specialty.upper()}:{chr(10)}[Resumen de hallazgos y recomendaciones]" for specialty in consultations.keys()])}

4. INTEGRACIÓN DIAGNÓSTICA
[Diagnósticos priorizados integrando todas las especialidades]

5. ÁREAS DE CONVERGENCIA
[Puntos donde los especialistas coinciden]

6. PLAN DE MANEJO INTEGRAL
[Plan coordinado que integra recomendaciones de todos los especialistas]
A) Tratamiento farmacológico integral
B) Estudios complementarios priorizados
C) Modificaciones al estilo de vida
D) Intervenciones específicas por especialidad

7. SEGUIMIENTO MULTIDISCIPLINARIO
[Plan de seguimiento coordinado]

8. PRONÓSTICO INTEGRAL
[Evaluación considerando todos los aspectos]

Dr. Roberto Mendoza García
Medicina Interna - Coordinador
Cédula: 1234567
"""
        return self.llm.predict(prompt)
    
    def _clean_json_response(self, response: str) -> str:
        """Limpia la respuesta para extraer JSON válido"""
        response = response.strip()
        if response.startswith("```json"):
            response = response[7:]
        if response.startswith("```"):
            response = response[3:]
        if response.endswith("```"):
            response = response[:-3]
        return response.strip()


class IntelligentMedicalConsultationSystem:
    """Sistema inteligente de interconsulta médica"""
    
    def __init__(self, api_key: str, model: str = "gpt-4"):
        self.llm = ChatOpenAI(
            temperature=0.7,
            model=model,
            openai_api_key=api_key
        )
        
        # Crear manager inteligente
        self.manager = IntelligentManagerAgent(self.llm)
        
        # Crear pool de especialistas disponibles
        self.available_specialists = {
            "neurologia": SpecialistAgent(self.llm, "neurologia", self._get_neurologist_prompt()),
            "cardiologia": SpecialistAgent(self.llm, "cardiologia", self._get_cardiologist_prompt()),
            "endocrinologia": SpecialistAgent(self.llm, "endocrinologia", self._get_endocrinologist_prompt()),
            "nefrologia": SpecialistAgent(self.llm, "nefrologia", self._get_nephrologist_prompt()),
        }
    
    def process_consultation(self, question: str, patient_record: str) -> Dict:
        """
        Procesa una consulta médica de manera inteligente
        
        Flujo:
        1. Manager analiza complejidad y decide si consultar especialistas
        2a. Si es simple: Manager responde directamente
        2b. Si es complejo: Consulta especialistas necesarios
        3. Manager integra (si hubo interconsultas)
        """
        results = {
            "decision_inicial": {},
            "resolucion_directa": None,
            "extracciones": {},
            "interconsultas": {},
            "integracion": None
        }
        
        print("\n" + "="*80)
        print("🏥 SISTEMA INTELIGENTE DE INTERCONSULTA MÉDICA")
        print("="*80)
        
        # FASE 1: Manager analiza y decide
        print("\n🧠 FASE 1: Análisis del Manager")
        print("-" * 80)
        print("El manager está analizando el caso...")
        print("Evaluando: complejidad, necesidad de especialistas, hallazgos críticos")
        
        decision = self.manager.analyze_case_complexity(question, patient_record)
        results["decision_inicial"] = decision
        
        print(f"\n✓ Análisis completado:")
        print(f"  Complejidad: {decision['complejidad'].upper()}")
        print(f"  Puede resolver solo: {'SÍ' if decision['puede_resolver_solo'] else 'NO'}")
        print(f"  Justificación: {decision['justificacion_decision']}")
        
        if decision['hallazgos_criticos']:
            print(f"  ⚠️  Hallazgos críticos detectados:")
            for hallazgo in decision['hallazgos_criticos']:
                print(f"     - {hallazgo}")
        
        # DECISIÓN: ¿Necesita especialistas?
        if decision['puede_resolver_solo']:
            # RUTA A: Manager resuelve solo
            print("\n" + "="*80)
            print("✅ DECISIÓN: El manager puede resolver el caso sin interconsultas")
            print("="*80)
            print("\n📝 Generando nota médica de medicina interna...\n")
            
            resolution = self.manager.resolve_as_internist(question, patient_record, decision)
            results["resolucion_directa"] = resolution
            
        else:
            # RUTA B: Necesita especialistas
            print("\n" + "="*80)
            print("⚠️  DECISIÓN: Se requieren interconsultas especializadas")
            print("="*80)
            
            specialists_needed = decision['especialistas_necesarios']
            print(f"\n👥 Especialistas a consultar: {', '.join(s.upper() for s in specialists_needed)}")
            
            for specialty in specialists_needed:
                reason = decision['razon_cada_especialista'].get(specialty, "Evaluación necesaria")
                print(f"\n   {specialty.upper()}: {reason}")
            
            # FASE 2: Extraer información relevante
            print("\n\n🔍 FASE 2: Extracción de información relevante")
            print("-" * 80)
            
            for specialty in specialists_needed:
                if specialty not in self.available_specialists:
                    print(f"⚠️  Especialista '{specialty}' no disponible, se omitirá")
                    continue
                    
                print(f"\n   Preparando consulta para {specialty.upper()}...")
                reason = decision['razon_cada_especialista'].get(specialty, "")
                
                specific_question, relevant_data = self.manager.extract_relevant_info(
                    question, patient_record, specialty, reason
                )
                
                results["extracciones"][specialty] = {
                    "pregunta": specific_question,
                    "datos": relevant_data,
                    "razon": reason
                }
                print(f"   ✓ Información extraída para {specialty}")
            
            # FASE 3: Consultar especialistas
            print("\n\n👨‍⚕️ FASE 3: Consultas especializadas")
            print("-" * 80)
            
            for specialty, extraction in results["extracciones"].items():
                print(f"\n   Consultando a {specialty.upper()}...")
                specialist = self.available_specialists[specialty]
                
                consultation = specialist.consult(
                    extraction["pregunta"],
                    extraction["datos"]
                )
                
                results["interconsultas"][specialty] = consultation
                print(f"   ✓ Interconsulta recibida")
            
            # FASE 4: Integración
            print("\n\n🏥 FASE 4: Integración por Medicina Interna")
            print("-" * 80)
            print("\n   El manager está integrando todas las interconsultas...")
            
            integration = self.manager.integrate_consultations(
                question,
                patient_record,
                decision,
                results["interconsultas"]
            )
            
            results["integracion"] = integration
            print("   ✓ Nota de integración completada")
        
        print("\n" + "="*80)
        print("✅ PROCESO COMPLETADO")
        print("="*80 + "\n")
        
        return results
    
    def format_full_report(self, results: Dict) -> str:
        """Genera reporte completo del proceso"""
        report = "="*80 + "\n"
        report += " SISTEMA INTELIGENTE DE INTERCONSULTA MÉDICA\n"
        report += "="*80 + "\n\n"
        
        # Mostrar decisión inicial
        decision = results["decision_inicial"]
        report += "🧠 ANÁLISIS INICIAL DEL MANAGER\n"
        report += "="*80 + "\n"
        report += f"Complejidad del caso: {decision['complejidad'].upper()}\n"
        report += f"Puede resolver solo: {'SÍ' if decision['puede_resolver_solo'] else 'NO'}\n\n"
        report += f"Justificación:\n{decision['justificacion_decision']}\n\n"
        
        if decision.get('hallazgos_criticos'):
            report += "⚠️  Hallazgos críticos:\n"
            for h in decision['hallazgos_criticos']:
                report += f"  • {h}\n"
            report += "\n"
        
        if decision['puede_resolver_solo']:
            # Mostrar resolución directa
            report += "\n" + "="*80 + "\n"
            report += "📋 RESOLUCIÓN DIRECTA POR MEDICINA INTERNA\n"
            report += "="*80 + "\n\n"
            report += results["resolucion_directa"] + "\n"
            
        else:
            # Mostrar proceso de interconsultas
            report += "\nEspecialistas consultados:\n"
            for specialty, reason in decision['razon_cada_especialista'].items():
                report += f"  • {specialty.upper()}: {reason}\n"
            
            # Distribución de información
            report += "\n\n" + "="*80 + "\n"
            report += "📋 INFORMACIÓN DISTRIBUIDA A ESPECIALISTAS\n"
            report += "="*80 + "\n"
            
            for specialty, extraction in results["extracciones"].items():
                report += f"\n{'─'*80}\n"
                report += f"➤ {specialty.upper()}\n"
                report += f"{'─'*80}\n"
                report += f"Razón de consulta: {extraction['razon']}\n\n"
                report += f"Pregunta específica:\n{extraction['pregunta']}\n\n"
                report += f"Datos proporcionados:\n{extraction['datos']}\n\n"
            
            # Interconsultas
            report += "\n" + "="*80 + "\n"
            report += "👨‍⚕️ INTERCONSULTAS ESPECIALIZADAS\n"
            report += "="*80 + "\n"
            
            for specialty, consultation in results["interconsultas"].items():
                report += f"\n{'█'*80}\n"
                report += f"  {specialty.upper()}\n"
                report += f"{'█'*80}\n\n"
                report += consultation + "\n\n"
            
            # Integración
            report += "\n" + "="*80 + "\n"
            report += "🏥 NOTA DE INTEGRACIÓN\n"
            report += "="*80 + "\n\n"
            report += results["integracion"] + "\n"
        
        return report
    
    # Prompts de especialistas
    def _get_neurologist_prompt(self) -> str:
        return """Eres el Dr. Carlos Ramírez, neurólogo certificado con 15 años de experiencia.

Solo recibirás información PERTINENTE a neurología.

FORMATO DE INTERCONSULTA - NEUROLOGÍA

MOTIVO DE INTERCONSULTA:
[Razón específica]

ANÁLISIS NEUROLÓGICO:
[Evaluación de síntomas y signos neurológicos]

IMPRESIÓN DIAGNÓSTICA:
1. [Diagnóstico principal]
2. [Diagnósticos diferenciales]

FUNDAMENTACIÓN:
[Justificación clínica]

RECOMENDACIONES:
1. [Manejo específico]
2. [Seguimiento]

ESTUDIOS SUGERIDOS:
- [Estudios neurológicos necesarios]

Dr. Carlos Ramírez - Neurología
Cédula: 3456789"""

    def _get_cardiologist_prompt(self) -> str:
        return """Eres la Dra. María Elena Torres, cardióloga certificada.

Solo recibirás información PERTINENTE a cardiología.

FORMATO DE INTERCONSULTA - CARDIOLOGÍA

MOTIVO DE INTERCONSULTA:
[Razón específica]

VALORACIÓN CARDIOVASCULAR:
[Análisis de factores de riesgo y hallazgos]

ESTRATIFICACIÓN DE RIESGO:
[Clasificación del riesgo cardiovascular]

IMPRESIÓN DIAGNÓSTICA:
1. [Diagnóstico principal]
2. [Comorbilidades]

RECOMENDACIONES:
1. [Manejo cardiovascular]
2. [Control de factores de riesgo]

ESTUDIOS SUGERIDOS:
- [Estudios cardiológicos]

Dra. María Elena Torres - Cardiología
Cédula: 4567890"""

    def _get_endocrinologist_prompt(self) -> str:
        return """Eres el Dr. Jorge Hernández, endocrinólogo certificado.

Solo recibirás información PERTINENTE a endocrinología.

FORMATO DE INTERCONSULTA - ENDOCRINOLOGÍA

MOTIVO DE INTERCONSULTA:
[Razón específica]

VALORACIÓN ENDOCRINOLÓGICA:
[Análisis metabólico y hormonal]

IMPRESIÓN DIAGNÓSTICA:
1. [Diagnóstico principal]
2. [Trastornos asociados]

RECOMENDACIONES:
1. [Manejo metabólico]
2. [Control glucémico/hormonal]

ESTUDIOS SUGERIDOS:
- [Estudios endocrinológicos]

Dr. Jorge Hernández - Endocrinología
Cédula: 5678901"""

    def _get_nephrologist_prompt(self) -> str:
        return """Eres la Dra. Patricia Morales, nefróloga certificada.

Solo recibirás información PERTINENTE a nefrología.

FORMATO DE INTERCONSULTA - NEFROLOGÍA

MOTIVO DE INTERCONSULTA:
[Razón específica]

VALORACIÓN NEFROLÓGICA:
[Análisis de función renal]

ESTADIFICACIÓN:
[Clasificación de ERC si aplica]

IMPRESIÓN DIAGNÓSTICA:
1. [Diagnóstico principal]
2. [Complicaciones]

RECOMENDACIONES:
1. [Nefroprotección]
2. [Seguimiento]

ESTUDIOS SUGERIDOS:
- [Estudios nefrológicos]

Dra. Patricia Morales - Nefrología
Cédula: 6789012"""


# ============================================================================
# EJEMPLO DE USO
# ============================================================================

if __name__ == "__main__":
    API_KEY = "tu-api-key-aqui"  # ⚠️ REEMPLAZAR
    system = IntelligentMedicalConsultationSystem(api_key=API_KEY, model="gpt-4")
    
    # CASO 1: Complejo - requiere especialistas
    caso_complejo = """
    Paciente con diagnóstico reciente de diabetes mellitus tipo 2.
    Presenta parestesias en miembros inferiores y reflejos aquíleos disminuidos.
    EKG con datos de hipertrofia ventricular izquierda.
    TFG de 68 mL/min con microalbuminuria.
    Requiero valoración integral.
    """
    
    expediente_complejo = """
PACIENTE: Masculino, 58 años
    
ANTECEDENTES:
- HTA 10 años, control irregular
- Tabaquismo 30 años (20 cig/día)
- Padre: DM2, IAM a los 62 años
- Madre: HAS, EVC a los 68 años

PADECIMIENTO ACTUAL (6 meses):
- Poliuria, polidipsia
- Pérdida de peso 8 kg
- Parestesias en pies (nocturnas)
- Visión borrosa
- Mareo ortostático

EXPLORACIÓN:
- TA: 150/95 mmHg, FC: 88 lpm
- IMC: 31.1 kg/m², PA: 108 cm
- Reflejos aquíleos +/++++
- Sensibilidad vibratoria abolida en ortejos
- Pulsos pedios +/++++
- Monofilamento: No percibe 5/10 puntos

LABORATORIOS:
- Glucosa: 187 mg/dL, HbA1c: 8.9%
- Colesterol: 245, LDL: 165, TG: 280
- Creatinina: 1.2, TFG: 68 mL/min
- Microalbuminuria: 120 mg/24h

EKG:
- Ritmo sinusal, FC 85
- HVI por voltaje
- Alteraciones de repolarización V5-V6
    """
    
    print("\n" + "🔬 CASO 1: CASO COMPLEJO")
    results1 = system.process_consultation(caso_complejo, expediente_complejo)
    report1 = system.format_full_report(results1)
    
    with open("caso_complejo.txt", "w", encoding="utf-8") as f:
        f.write(report1)
    
    print("\n" + "="*80)
    print("CASO COMPLEJO - Guardado en: caso_complejo.txt")
    print("="*80)
    
    # CASO 2: Simple - manager puede resolver
    caso_simple = """
    Paciente con cefalea tensional recurrente.
    Sin datos de alarma neurológica.
    ¿Requiere estudios de imagen o puedo iniciar manejo sintomático?
    """
    
    expediente_simple = """
PACIENTE: Femenino, 32 años

ANTECEDENTES:
- Previamente sana
- Estrés laboral crónico

PADECIMIENTO ACTUAL (3 meses):
- Cefalea holocraneana opresiva
- Intensidad 5/10
- Aumenta con estrés
- Mejora con analgésicos
- Sin náusea, vómito, fotofobia
- Sin alteraciones visuales

EXPLORACIÓN:
- Signos vitales normales
- Neurológico: sin datos focales
- Fondo de ojo: normal
- Reflejos normales

LABORATORIOS:
- BH, QS: normales
    """
    
    print("\n\n" + "🔬 CASO 2: CASO SIMPLE")
    results2 = system.process_consultation(caso_simple, expediente_simple)
    report2 = system.format_full_report(results2)
    
    with open("caso_simple.txt", "w", encoding="utf-8") as f:
        f.write(report2)
    
    print("\n" + "="*80)
    print("CASO SIMPLE - Guardado en: caso_simple.txt")
    print("="*80)